<h1><center>Coleridge: Complete EDA in ONE Notebook</center></h1>

<center><img src = "https://coleridgeinitiative.org/wp-content/uploads/2020/04/small-coleridge_logo_modern_stacked-black.png" width = "350" height = "250"/></center>             

                                                                                   

The [Coleridge Initiative](https://coleridgeinitiative.org/) is a not-for-profit organization, originally established at New York University, that is working with governments to ensure that data are more effectively used for public decision-making. We achieve this goal by working with government agencies to create value for the taxpayer from the careful use of data, by building new technologies to enable secure access to and sharing of confidential microdata, and by training agency staff to acquire modern data skills.

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

1. [Competition Overview](#competition-overview)
2. [Preliminaries](#preliminaries)
3. [Load Datasets](#load-datasets)
4. [Tabular Exploration](#tabular-exploration)
5. [Publication Title Exploration](#publication-title-exploration)
6. [Dataset Title Exploration](#dataset-title-exploration)
7. [Dataset Label Exploration](#dataset-label-exploration)
8. [Model](#model)

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. </center>                                                   <center>This notebook will be updated frequently so keep checking for furthur developments.</center></h2>

#### If you would have any doubts or need guidance in Data Science you can reach out to me on [LinkedIn](https://www.linkedin.com/in/ishandutta0098)

<a id="competition-overview"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Competition Overview</center></h2>

### Description

In this competition, you'll use natural language processing (NLP) to automate the discovery of how scientific data are referenced in publications. Utilizing the full text of scientific publications from numerous research areas gathered from CHORUS publisher members and other sources, you'll identify data sets that the publications' authors used in their work.

If successful, you'll help support evidence in government data. Automated NLP approaches will enable government agencies and researchers to quickly find the information they need. The approach will be used to develop data usage scorecards to better enable agencies to show how their data are used and bring down a critical barrier to the access and use of public data.

### Evaluation Criteria
The objective of the competition is to identify the mention of datasets within scientific publications. Your predictions will be short excerpts from the publications that appear to note a dataset.

Submissions are evaluated on a **Jaccard-based FBeta score** between predicted texts and ground truth texts, with `Beta = 0.5` (a `micro F0.5` score). Multiple predictions are delineated with a pipe (|) character in the submission file.

The following is Python code for calculating the Jaccard score for a single prediction string against a single ground truth string. Note that the overall score for a sample uses Jaccard to compare multiple ground truth and prediction strings that are pipe-delimited - this code does not handle that process or the final `micro F-beta` calculation.

### Code Requirements

<img src = "https://i.imgur.com/S32GCaA.png" width = "600" height = "250"/>  

Submissions to this competition must be made through Notebooks. In order for the "Submit" button to be active after a commit, the following conditions must be met:

- **CPU Notebook** <= 9 hours run-time
- **GPU Notebook** <= 9 hours run-time
- **Internet** access disabled
- Freely & publicly available **external data** is allowed, including pre-trained models
- **Submission file** must be named submission.csv

### Data Description
The objective of the competition is to identify the mention of datasets within scientific publications. Your predictions will be short excerpts from the **publications** that appear to note a dataset. Predictions that more accurately match the precise words used to identify the dataset within the publication will score higher. Predictions should be cleaned using the `clean_text` function from the [Evaluation page](https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/overview/evaluation) to ensure proper matching.

Publications are provided in **JSON format**, broken up into sections with section titles.

The goal in this competition is not just to match known dataset strings but to generalize to datasets that have never been seen before using **NLP** and statistical techniques. A percentage of the public test set publications are drawn from the training set - not all datasets have been identified in train, so these unidentified datasets have been used as a portion of the public test labels. These should serve as guides for the difficult task of labeling the private test set.

Note that the hidden test set has roughly ~8000 publications, many times the size of the public test set. Plan your compute time accordingly.

### Files
- **train** - the full text of the training set's publications in JSON format, broken into sections with section titles
- **test** - the full text of the test set's publications in JSON format, broken into sections with section titles
- **train.csv** - labels and metadata for the training set
- **sample_submission.csv** - a sample submission file in the correct format

### Columns
- `id` - publication id - note that there are multiple rows for some training documents, indicating multiple mentioned datasets
- `pub_title` - title of the publication (a small number of publications have the same title)
- `dataset_title` - the title of the dataset that is mentioned within the publication
- `dataset_label` - a portion of the text that indicates the dataset
- `cleaned_label` - the dataset_label, as passed through the `clean_text` function from the [Evaluation page](https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/overview/evaluation)

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases (W&B)</center></h2>

<center><img src = "https://i.imgur.com/1sm6x8P.png" width = "750" height = "500"/></center>  

**Weights & Biases** is the machine learning platform for developers to build better models faster. 

You can use W&B's lightweight, interoperable tools to 
- quickly track experiments, 
- version and iterate on datasets, 
- evaluate model performance, 
- reproduce models, 
- visualize results and spot regressions, 
- and share findings with colleagues. 

Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In this notebook I will use Weights and Biases's amazing features to perform wonderful visualizations and logging seamlessly. 

<a id="preliminaries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Preliminaries</center></h2>

In [ ]:
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import cv2
from wordcloud import WordCloud, STOPWORDS

#Text Color
from termcolor import colored

#Data Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

#NLP
from sklearn.feature_extraction.text import CountVectorizer

#WordCloud
from wordcloud import WordCloud, STOPWORDS

#Text Processing
import nltk
nltk.download('popular')

#Language Detection
!pip install langdetect
import langdetect

#Sentiment
from textblob import TextBlob

#ner
import spacy

#Vectorizer
from sklearn import feature_extraction, manifold

#Word Embedding
import gensim.downloader as gensim_api

#Topic Modeling
import gensim

# HTML
from IPython.core.display import HTML

import re
import os
import gc
import glob
import json
import time
import torch
import random
import datetime
import tokenizers
import numpy as np
import transformers
import pandas as pd
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt

from tokenizers import *
from transformers import *
from functools import partial
from tqdm.notebook import tqdm
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import GroupKFold

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

We begin with loading the dataset to work upon. Here we have one `.csv` file which has been loaded,

In [ ]:
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

<a id="tabular-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Tabular Exploration</center></h2>

### Dataset Head and Info

Let us quickly have a brief look upon the dataset. Hope to not encounter any NULL values!

In [ ]:
train.head()

In [ ]:
train.info()

Aha! There are no NULL values, so we are good to go.

### Dataset Size

In [ ]:
print(f"Training Dataset Shape: {colored(train.shape, 'yellow')}")

### Column-wise Unique Values

In [ ]:
for col in train.columns:
    print(col + ":" + colored(str(len(train[col].unique())), 'yellow'))

- The Training Dataset has **19,661** samples but it looks like there are only **14,316** unique IDs in the dataset, meaning that some publications include a multitude of datasets. 
- Also, notice that the `pub_title` unique count is slightly smaller than the `Id` unique counts. This points to the precense of several occurences of having 2 separate publications, each with a unique ID, but sharing the exact same title.
- Also, there are a total of **45 unique** `dataset_title` and **130** `unique dataset_label`. Meaning that a single dataset could have multible labels throughout different publications.

From here onwards for the next part, we shall have a little deeper look at the dataset. We move ahead trying to understand each column one by one.

<a id="publication-title-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Publication Title Exploration</center></h2>

<img src = "https://lh3.googleusercontent.com/proxy/mrCb0ve5T8GS4NZteL9laHaTvMM9EtJz9OBnrvx1vfkqeokKZLYcmnOiI_jCt2zpUpY2G03xITWWMqHHLKnvbnsTGaw" width = 600 height = 250 />

In [ ]:
train['pub_title']

### Wordcloud

Word Cloud is a data visualization technique used for representing text data in which the size of each word indicates its frequency or importance. Significant textual data points can be highlighted using a word cloud. Word clouds are widely used for analyzing data from social network websites.

We will write a simple and intuitive function `plot_wordcloud` that will help us plot wordclouds with ease. 

In [ ]:
def plot_wordcloud(column, title):
    
    """
    Function to Plot Wordcloud of given dataframe column.
    
    params: column(string): The Column of the DataFrame for plotting.
            title(string) : The Title of the Wordcloud.
    """
    # Define stopwords
    stopwords = set(STOPWORDS) 
    
    # Define the Wordcloud    
    wordcloud = WordCloud(width = 800, 
                          height = 800,
                          background_color ='black',
                          min_font_size = 10,
                          stopwords = stopwords).generate(' '.join(train[column])) 

    # Plot the WordCloud image                        
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title('Wordcloud: ' + title, fontsize = 20)

    plt.show()  

In [ ]:
plot_wordcloud(column = 'pub_title', title = 'Publication Title')

### Basic NLP

To understand the column better we need to clean it first. For this purpose we write another intuitive function `preprocess_text`.

In [ ]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True):
    
    """
    Function to Preprocess the Text by removing extra spaces and stopwords.
    It also performs Stemming or Lemmatization.
    
    params: text(string)   : The Text which has to be cleaned.
            flg_stemm(bool): Flag for Stemming
            flg_lemm(bool) : Flag for Lemmatization
            
    returns: text(string)  : Returns the Cleaned Text
    """

    lst_stopwords = nltk.corpus.stopwords.words("english")
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()    
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

Now that we have our function, let's clean the column and calculate some stats!

In [ ]:
#Clean Publication Title
train["clean_pub_title"] = train["pub_title"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, ))

#Length
train['clean_pub_title_len'] = train['clean_pub_title'].apply(lambda x: len(x))

#Word Count
train['clean_pub_title_word_count'] =train["clean_pub_title"].apply(lambda x: len(str(x).split(" ")))

#Character Count
train['clean_pub_title_char_count'] = train["clean_pub_title"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#Average Word Length
train['clean_pub_title_avg_word_length'] = train['clean_pub_title_char_count'] / train['clean_pub_title_word_count']

### Distribution Plots

A good way to understand the distribution of our calculated stats is by plotting a distribution plot. We again write a function `plot_distribution` which will save us a lot of time.

In [ ]:
def plot_distribution(x, title):
    
    """
    Function to obtain the distribution plot of given data.
    
    params: x(string)     : Name of the Column for the Plot.
            title(string) : Title of the Plot
    """
    sns.displot(train, x = x, kind="kde", bw_adjust=2)

    plt.title(title, fontsize = 15)
    plt.show()

Instead of calling the function 4 times, we imporovise and make a list of tuples. These tuples of the form (column, title) contain the inputs of the function `plot_distribution`.

In [ ]:
# List of Tuples for Plot Distribution

pub_title_list = [("clean_pub_title_len", "Publication Title: Length Distribution"),
                 ("clean_pub_title_word_count", "Publication Title: Word Count Distribution"),
                 ("clean_pub_title_char_count", "Publication Title: Character Count Distribution"),
                 ("clean_pub_title_avg_word_length", "Publication Title: Average Word Length Distribution")]

In [ ]:
for column, title in pub_title_list:
    plot_distribution(x = column, title = title)    

<a id="dataset-title-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Dataset Title Exploration</center></h2>

<img src = "https://marketinginapod.files.wordpress.com/2018/03/bigdata-e1520528391656.jpg?w=377&h=206" width = 600 height = 250 />

In [ ]:
train['dataset_title']

### Wordcloud

In [ ]:
plot_wordcloud(column = 'dataset_title', title = 'Dataset Title')

### Basic NLP

In [ ]:
#Clean Dataset Title
train["clean_dataset_title"] = train["dataset_title"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, ))

#Length
train['clean_dataset_title_len'] = train['clean_dataset_title'].apply(lambda x: len(x))

#Word Count
train['clean_dataset_title_word_count'] =train["clean_dataset_title"].apply(lambda x: len(str(x).split(" ")))

#Character Count
train['clean_dataset_title_char_count'] = train["clean_dataset_title"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#Average Word Length
train['clean_dataset_title_avg_word_length'] = train['clean_dataset_title_char_count'] / train['clean_dataset_title_word_count']

### Distribution Plots

In [ ]:
# List of Tuples for Plot Distribution

dataset_title_list = [("clean_dataset_title_len", "Dataset Title: Length Distribution"),
                     ("clean_dataset_title_word_count", "Dataset Title: Word Count Distribution"),
                     ("clean_dataset_title_char_count", "Dataset Title: Character Count Distribution"),
                     ("clean_dataset_title_avg_word_length", "Dataset Title: Average Word Length Distribution")]

In [ ]:
for column, title in dataset_title_list:
    plot_distribution(x = column, title = title)

<a id="dataset-label-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Dataset Label Exploration</center></h2>

In [ ]:
train['dataset_label']

### Wordcloud

In [ ]:
plot_wordcloud(column = 'dataset_label', title = 'Dataset Label')

### Basic NLP

In [ ]:
#Clean Dataset Label
train["clean_dataset_label"] = train["dataset_label"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, ))

#Length
train['clean_dataset_label_len'] = train['clean_dataset_label'].apply(lambda x: len(x))

#Word Count
train['clean_dataset_label_word_count'] =train["clean_dataset_label"].apply(lambda x: len(str(x).split(" ")))

#Character Count
train['clean_dataset_label_char_count'] = train["clean_dataset_label"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#Average Word Length
train['clean_dataset_label_avg_word_length'] = train['clean_dataset_label_char_count'] / train['clean_dataset_label_word_count']

### Distribution Plots

In [ ]:
# List of Tuples for Plot Distribution

dataset_label_list = [("clean_dataset_label_len", "Dataset Label: Length Distribution"),
                     ("clean_dataset_label_word_count", "Dataset Label: Word Count Distribution"),
                     ("clean_dataset_label_char_count", "Dataset Label: Character Count Distribution"),
                     ("clean_dataset_label_avg_word_length", "Dataset Label: Average Word Length Distribution")]

In [ ]:
for column, title in dataset_label_list:
    plot_distribution(x = column, title = title)

<a id="model"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Model</center></h2>

## Imports

In [ ]:
import wandb
wandb.login()

## Params

In [ ]:
SEED = 2020

DATA_PATH = "../input/coleridgeinitiative-show-us-the-data/"
DATA_PATH_TRAIN = DATA_PATH + 'train/'
DATA_PATH_TEST = DATA_PATH + 'test/'

NUM_WORKERS = 4

VOCABS = {
    "bert-base-uncased": "../input/vocabs/bert-base-uncased-vocab.txt",
}

MODEL_PATHS = {
    'bert-base-uncased': '../input/bertconfigs/uncased_L-12_H-768_A-12/uncased_L-12_H-768_A-12/',
    'bert-large-uncased-whole-word-masking-finetuned-squad': '../input/bertconfigs/wwm_uncased_L-24_H-1024_A-16/wwm_uncased_L-24_H-1024_A-16/',
    'albert-large-v2': '../input/albert-configs/albert-large-v2/albert-large-v2/',
    'albert-base-v2': '../input/albert-configs/albert-base-v2/albert-base-v2/',
    'distilbert': '../input/albert-configs/distilbert/distilbert/',
}

In [ ]:
# wandb config
WANDB_CONFIG = {
     'competition': 'Coleridge', 
              '_wandb_kernel': 'neuracort'
    }

## Make Data

In [ ]:
def load_text(id_, root=""):
    with open(os.path.join(root, id_ + ".json")) as f:
        text = json.load(f)
    return text


def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()


def create_data():
    new_df = []

    for idx in tqdm(range(len(df))):
        article = load_text(df['Id'][idx], DATA_PATH_TRAIN)
        id_, pub_title, dataset_title, dataset_label, cleaned_label = df.iloc[idx]
        
        
        for i, section in enumerate(article):
            text = section['text']
            title = section['section_title']
            
            cleaned_text = clean_text(section['text'])
            
            found = cleaned_label in cleaned_text
    
            dic = {
                "id": [id_], 
                "section_id": [i],
                "pub_title": [pub_title], 
                "dataset_title": [dataset_title], 
                "dataset_label": [dataset_label], 
                "cleaned_label": [cleaned_label],
                "text": [text],
                "cleaned_text": [cleaned_text],
                "label_found": [found],
            }
            new_df.append(pd.DataFrame.from_dict(dic))
            
    return pd.concat(new_df).reset_index(drop=True)

In [ ]:
df = pd.read_csv(DATA_PATH + 'train.csv')

new_df = create_data()

In [ ]:
df = new_df[new_df['label_found']].reset_index(drop=True)

df['length'] = df['cleaned_text'].apply(lambda x: len(x.split()))
df = df[df['length'] < 3000]  # remove too long texts

df.to_csv("df_train.csv", index=False)  # saving, just in case

## Utils

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results.

    Args:
        seed (int): Number of the seed.
    """

    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def save_model_weights(model, filename, verbose=1, cp_folder=""):
    """
    Saves the weights of a PyTorch model.

    Args:
        model (torch model): Model to save the weights of.
        filename (str): Name of the checkpoint.
        verbose (int, optional): Whether to display infos. Defaults to 1.
        cp_folder (str, optional): Folder to save to. Defaults to "".
    """

    if verbose:
        print(f"\n -> Saving weights to {os.path.join(cp_folder, filename)}\n")
    torch.save(model.state_dict(), os.path.join(cp_folder, filename))


def count_parameters(model, all=False):
    """
    Count the parameters of a model.

    Args:
        model (torch model): Model to count the parameters of.
        all (bool, optional):  Whether to count not trainable parameters. Defaults to False.

    Returns:
        int: Number of parameters.
    """

    if all:
        return sum(p.numel() for p in model.parameters())
    else:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Data

In [ ]:
class EncodedText:
    def __init__(self, ids, offsets):
        self.ids = ids
        self.offsets = offsets


def create_tokenizer_and_tokens(config):
    if "roberta" in config.selected_model:
        raise NotImplementedError
        
    elif "albert" in config.selected_model:
        raise NotImplementedError
        
    else:
        tokenizer = BertWordPieceTokenizer(
            MODEL_PATHS[config.selected_model] + 'vocab.txt',
            lowercase=config.lowercase,
        )

        tokens = {
            'cls': tokenizer.token_to_id('[CLS]'),
            'sep': tokenizer.token_to_id('[SEP]'),
            'pad': tokenizer.token_to_id('[PAD]'),
        }
    
    return tokenizer, tokens

def locate_label_string(text, label):
    """
    Finds the label in the text
    """
    len_label = len(label) - 1

    candidates_idx = [i for i, e in enumerate(text) if e == label[1]]
    for idx in candidates_idx:
        if " " + text[idx: idx + len_label] == label:
            idx_start = idx
            idx_end = idx + len_label
            break

    assert (
        text[idx_start:idx_end] == label[1:]
    ), f'"{text[idx_start: idx_end]}" instead of "{label}" in "{text}"'

    char_targets = np.zeros(len(text))
    char_targets[idx_start:idx_end] = 1

    return idx_start, idx_end, char_targets


def locate_label_tokens(offsets, char_targets):
    """
    Finds the tokens corresponding to the found labels
    """
    target_idx = []
    for idx, (offset1, offset2) in enumerate(offsets):
        if sum(char_targets[offset1:offset2]) > 0:
            target_idx.append(idx)

    if not len(target_idx):
        for idx, (offset1, offset2) in enumerate(offsets):
            if sum(char_targets[offset1:offset2]) > 0:
                target_idx.append(idx)

    return target_idx[0], target_idx[-1]

def process_data(
    text,
    label,
    tokenizer,
    tokens,
    max_len=100,
    model_name="bert",
):
    """
    Prepares the data for the question answering task.
    Adapted from Abishek's work on the Tweet Sentiment extraction competition, 
    check his work for more details !
    """
    target_start, target_end = 0, 0
    text = " " + " ".join(str(text).split())
    label = " " + " ".join(str(label).split())

    if label != " ":
        idx_start, idx_end, char_targets = locate_label_string(
            text, label
        )

    tokenized = tokenizer.encode(text)
    input_ids_text = tokenized.ids[1:-1]

    # print(input_ids_text, len(input_ids_text))

    offsets = tokenized.offsets[1:-1]

    if label != " ":
        target_start, target_end = locate_label_tokens(offsets, char_targets)

    if target_end >= max_len - 2:  # target is too far in the sentence, we crop its beginning.
        n_tok_to_crop = target_start - max_len // 2
        new_str_start = offsets[n_tok_to_crop][0]

        input_ids_text = input_ids_text[n_tok_to_crop:]

        offsets = [tuple(t) for t in np.array(offsets[n_tok_to_crop:]) - new_str_start]
        text = text[new_str_start:]

        target_start -= n_tok_to_crop
        target_end -= n_tok_to_crop

    input_ids = (
        [tokens["cls"]]
        + input_ids_text[:max_len - 2]
        + [tokens["sep"]]
    )

    if "roberta" in model_name:
        token_type_ids = [0] * len(input_ids)
    else:
        token_type_ids = [1] * len(input_ids)

    text_offsets = [(0, 0)] + offsets[:max_len - 2] + [(0, 0)]

    target_start += 1
    target_end += 1

    # target_end = min(target_end, max_len - 1)

    assert len(input_ids) == len(token_type_ids) and len(input_ids) == len(text_offsets), (len(input_ids), len(text_offsets))  # noqa

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([tokens["pad"]] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        text_offsets = text_offsets + ([(0, 0)] * padding_length)

    return {
        "ids": input_ids,
        "token_type_ids": token_type_ids,
        "targets_start": target_start,
        "targets_end": target_end,
        "text": text,
        "label": label,
        "offsets": text_offsets,
    }

## Dataset

In [ ]:
class SectionDataset(Dataset):
    def __init__(
        self,
        df,
        tokenizer,
        tokens,
        max_len=512,
        model_name="bert",
    ):
        self.tokens = tokens
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.model_name = model_name

        self.texts = df["cleaned_text"].values
        self.labels = df["cleaned_label"].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        data = process_data(
            self.texts[idx],
            self.labels[idx],
            self.tokenizer,
            self.tokens,
            max_len=self.max_len,
            model_name=self.model_name,
        )

        return {
            "ids": torch.tensor(data["ids"], dtype=torch.long),
            "token_type_ids": torch.tensor(data["token_type_ids"], dtype=torch.long),
            "target_start": torch.tensor(data["targets_start"], dtype=torch.long),
            "target_end": torch.tensor(data["targets_end"], dtype=torch.long),
            "text": data["text"],
            "label": data["label"],
            "offsets": torch.tensor(data["offsets"], dtype=torch.long),
        }

## Model

In [ ]:
TRANSFORMERS = {
    "roberta-base": (RobertaModel, "roberta-base"),
    'albert-base-v2': (AlbertModel, 'albert-base-v2'),
    'albert-large-v2': (AlbertModel, 'albert-large-v2'),
    'albert-xlarge-v2': (AlbertModel, 'albert-xlarge-v2'),
    'albert-xxlarge-v2': (AlbertModel, 'albert-xxlarge-v2'),
    "bert-base-uncased": (BertModel, "bert-base-uncased"),
    "bert-base-cased": (BertModel, "bert-base-cased"),
    "bert-large-uncased-whole-word-masking": (BertModel, "bert-large-uncased-whole-word-masking"),
    "distilbert-base-uncased-distilled-squad": (
        DistilBertModel,
        "distilbert-base-uncased-distilled-squad"
    )
}


class QATransformer(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.name = model

        self.pad_idx = 1 if "roberta" in self.name else 0

        model_class, pretrained_weights = TRANSFORMERS[model]

        self.transformer = model_class.from_pretrained(
            pretrained_weights, output_hidden_states=True
        )

        self.nb_features = self.transformer.pooler.dense.out_features

        self.logits = nn.Sequential(
            nn.Linear(self.nb_features, self.nb_features),
            nn.Tanh(),
            nn.Linear(self.nb_features, 2),
        )

    def forward(self, tokens, token_type_ids):
        """
        Usual torch forward function

        Arguments:
            tokens {torch tensor} -- Sentence tokens
            token_type_ids {torch tensor} -- Sentence tokens ids
        """

        hidden_states = self.transformer(
            tokens,
            attention_mask=(tokens != self.pad_idx).long(),
            token_type_ids=token_type_ids,
        )[-1]

        features = hidden_states[-1]
        logits = self.logits(features)

        start_logits, end_logits = logits[:, :, 0], logits[:, :, 1]

        return start_logits, end_logits

## Metric

In [ ]:
def jaccard_similarity(s1, s2): # could be wrong, see CPMP's thread
    l1 = s1.split(" ")
    l2 = s2.split(" ")
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union


def compute_score(y_true, y_pred, beta=0.5):
    """
    From https://www.kaggle.com/tungmphung/coleridge-initiative-local-score-computation/
    """
    TP, FP, FN = 0, 0, 0

    for truth, pred in zip(y_true, y_pred):
        true_datasets = truth.split('|')
        # Predicted strings for each publication are sorted alphabetically
        # and processed in that order.
        pred_datasets = sorted(pred.split('|'))

        for true_dataset in true_datasets:
            if len(pred_datasets):
                match_scores = [jaccard_similarity(true_dataset, pred_dataset)
                                for pred_dataset in pred_datasets]
                # The prediction with the highest score for a given ground truth
                # is matched with that ground truth.
                match_index = np.argmax(match_scores)

                if match_scores[match_index] >= 0.5:
                    # Any matched predictions where the Jaccard score meets or
                    # exceeds the threshold of 0.5 are counted as true positives (TP),
                    TP += 1
                else:
                    # the remainder as false positives (FP).
                    FP += 1

                del(pred_datasets[match_index])
            else:
                # Any ground truths with no nearest predictions are counted as
                # false negatives (FN).
                FN += 1
        # Any unmatched predictions are counted as false positives (FP).
        FP += len(pred_datasets)

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = (1 + beta**2)*(precision*recall)/((beta**2)*precision + recall)

    return f_score

In [ ]:
def get_string_from_idx(text, idx_start, idx_end, offsets):
    """
    Uses the offsets to retrieve the predicted string based on the start and end indices
    """
    if idx_end < idx_start:
        idx_end = idx_start

    predicted_string = ""
    for i in range(idx_start, idx_end + 1):
        predicted_string += text[offsets[i][0]: offsets[i][1]]
        if i + 1 < len(offsets) and offsets[i][1] < offsets[i + 1][0]:
            predicted_string += " "

    return predicted_string

def get_pred_from_logits(data, start_logits, end_logits, from_proba=False):
    if not from_proba:
        start_logits = torch.softmax(start_logits, dim=1).cpu().detach().numpy()
        end_logits = torch.softmax(end_logits, dim=1).cpu().detach().numpy()

    offsets = data["offsets"].cpu().numpy()

    preds = []
    for i in range(len(start_logits)):
        start_idx = np.argmax(start_logits[i])
        end_idx = np.argmax(end_logits[i])
        preds.append(get_string_from_idx(data["text"][i], start_idx, end_idx, offsets[i]))

    return preds

In [ ]:
def ce_loss(
    pred, truth, smoothing=False, trg_pad_idx=-1, eps=0.1
):
    """
    Computes the cross entropy loss with label smoothing

    Args:
        pred (torch tensor): Prediction
        truth (torch tensor): Target
        smoothing (bool, optional): Whether to use smoothing. Defaults to False.
        trg_pad_idx (int, optional): Indices to ignore in the loss. Defaults to -1.
        eps (float, optional): Smoothing coefficient. Defaults to 0.1.

    Returns:
        [type]: [description]
    """
    truth = truth.contiguous().view(-1)

    one_hot = torch.zeros_like(pred).scatter(1, truth.view(-1, 1), 1)

    if smoothing:
        n_class = pred.size(1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)

    loss = -one_hot * F.log_softmax(pred, dim=1)

    if trg_pad_idx >= 0:
        loss = loss.sum(dim=1)
        non_pad_mask = truth.ne(trg_pad_idx)
        loss = loss.masked_select(non_pad_mask)

    return loss.sum()


def qa_loss_fn(start_logits, end_logits, start_positions, end_positions, config):
    """
    Loss function for the question answering task.
    It is the sum of the cross entropy for the start and end logits

    Args:
        start_logits (torch tensor): Start logits
        end_logits (torch tensor): End logits
        start_positions (torch tensor): Start ground truth
        end_positions (torch tensor): End ground truth
        config (dict): Dictionary of parameters for the CE Loss.

    Returns:
        torch tensor: Loss value
    """
    bs = start_logits.size(0)

    start_loss = ce_loss(
        start_logits,
        start_positions,
        smoothing=config["smoothing"],
        eps=config["eps"],
    )

    end_loss = ce_loss(
        end_logits,
        end_positions,
        smoothing=config["smoothing"],
        eps=config["eps"],
    )

    total_loss = start_loss + end_loss

    return total_loss / bs

In [ ]:
def trim_tensors(tokens, input_ids, model_name='bert', min_len=10):
    """
    Trim tensors so that within a batch, padding is shortened.
    This speeds up training for RNNs and Transformers

    Arguments:
        tokens {torch tensor} -- Text tokens

    Keyword Arguments:
        min_len {int} -- Minimum length to trim to (default: {10})

    Returns:
        torch tensor -- trimmed tokens
    """
    pad_token = 1 if "roberta" in model_name else 0
    max_len = max(torch.max(torch.sum((tokens != pad_token), 1)), min_len)
    return tokens[:, :max_len], input_ids[:, :max_len]

## Fit

In [ ]:
def fit(
    model,
    train_dataset,
    val_dataset,
    loss_config,
    epochs=5,
    batch_size=8,
    weight_decay=0,
    warmup_prop=0.0,
    lr=5e-4,
):

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, num_workers=NUM_WORKERS, shuffle=True,
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS
    )

    opt_params = []
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    for n, p in model.named_parameters():
        wd = 0 if any(nd in n for nd in no_decay) else weight_decay
        opt_params.append(
            {"params": [p], "weight_decay": wd, "lr": lr}
        )

    optimizer = AdamW(opt_params, lr=lr, betas=(0.5, 0.999))

    n_steps = epochs * len(train_loader)
    num_warmup_steps = int(warmup_prop * n_steps)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps, n_steps
    )

    # Initialize W&B
    run = wandb.init(project='Coleridge-BERT', config= WANDB_CONFIG)
    
    total_steps = 0
    for epoch in range(epochs):
        model.train()
        start_time = time.time()

        optimizer.zero_grad()
        avg_loss = 0

        for step, data in enumerate(train_loader):
            total_steps += 1

            ids, token_type_ids = trim_tensors(
                data["ids"], data["token_type_ids"], model.name
            )

            start_logits, end_logits = model(ids.cuda(), token_type_ids.cuda())

            loss = qa_loss_fn(
                start_logits,
                end_logits,
                data["target_start"].cuda(),
                data["target_end"].cuda(),
                config=loss_config,
            )
            
            wandb.log({'loss': loss})

            avg_loss += loss.item() / len(train_loader)
            loss.backward()

            nn.utils.clip_grad_norm_(model.parameters(), 10.0)

            optimizer.step()
            scheduler.step()
            model.zero_grad()
            
        # Close W&B run
        wandb.finish()

        model.eval()
        avg_val_loss = 0.
        preds, truths = [], []

        with torch.no_grad():

            for data in val_loader:
                ids, token_type_ids = trim_tensors(
                    data["ids"], data["token_type_ids"], model.name
                )

                start_logits, end_logits = model(ids.cuda(), token_type_ids.cuda())

                loss = qa_loss_fn(
                    start_logits.detach(),
                    end_logits.detach(),
                    data["target_start"].cuda(),
                    data["target_end"].cuda(),
                    config=loss_config,
                )

                avg_val_loss += loss.item() / len(val_loader)

                preds += get_pred_from_logits(data, start_logits, end_logits)
                truths += data['label']

        score = compute_score(truths, preds)

        dt = time.time() - start_time
        lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch + 1}/{epochs} \t lr={lr:.1e} \t t={dt:.0f}s \t", end="")
        print(
            f"loss={avg_loss:.3f} \t val_loss={avg_val_loss:.3f} \t val_score={score:.4f}"
        )

    del loss, data, avg_val_loss, avg_loss, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()

    return preds

In [ ]:
def k_fold(config, df, save=True):
    tokenizer, tokens = create_tokenizer_and_tokens(config)
    
    time = re.sub(' ', '_', str(datetime.datetime.now())[:16])
    score = 0
    
    gkf = GroupKFold(n_splits=config.k)
    folds = list(gkf.split(X=df, groups=df['dataset_title']))
    
    pred_oof = [''] * len(df)
    
    for fold, (train_idx, val_idx) in enumerate(folds):
        if fold in config.selected_folds:
            print(f"\n-------------   Fold {fold + 1} / {len(folds)}  -------------\n")
            seed_everything(config.seed + fold)

            model = QATransformer(config.selected_model).cuda()
            model.zero_grad()

            train_dataset = SectionDataset(
                df.iloc[train_idx], 
                tokenizer, 
                tokens, 
                max_len=config.max_len, 
                model_name=config.selected_model
            )

            val_dataset = SectionDataset(
                df.iloc[val_idx], 
                tokenizer, 
                tokens,
                max_len=config.max_len, 
                model_name=config.selected_model
            )

            n_parameters = count_parameters(model)

            print(f"    -> {len(train_dataset)} training texts")
            print(f"    -> {len(val_dataset)} validation texts")
            print(f"    -> {n_parameters} trainable parameters\n")  

            preds = fit(
                model, 
                train_dataset, 
                val_dataset, 
                config.loss_config,
                epochs=config.epochs, 
                batch_size=config.batch_size, 
                weight_decay=config.weight_decay, 
                lr=config.lr, 
                warmup_prop=config.warmup_prop,
            )

            for i, idx in enumerate(val_idx):
                pred_oof[idx] = preds[i]

            if save:
                save_model_weights(model, f'{config.selected_model}_{fold + 1}.pt', cp_folder="")

            del model, train_dataset, val_dataset
            torch.cuda.empty_cache()
            gc.collect()

In [ ]:
class Config:
    # General
    k = 5
    seed = 2021
    selected_folds = [0]

    # Texts
    max_len = 256
    
    # Architecture
    selected_model = "bert-base-uncased"
    lowercase = True
    
    # Loss function
    loss_config = {
        "smoothing": False,
        "eps": 0.1,
    }
    
    # Training
    batch_size = 16
    batch_size_val = batch_size * 2
    weight_decay = 1.
    
    epochs = 1
    lr = 5e-5
    warmup_prop = 0.1

In [ ]:
df = pd.read_csv("df_train.csv")

k_fold(
    Config,
    df,
    save=True,
)

[WandB Project Page](https://wandb.ai/ishandutta/Coleridge-BERT?workspace=user-ishandutta)